In [35]:
import pandas as pd
import numpy as np

# Загрузка данных
df = pd.read_csv('LizaAlert.csv')

# Проверка наличия столбца status
if 'status' not in df.columns:
    print("Ошибка: столбец 'status' не найден")
else:
    # Приводим статус к строке и очищаем от пробелов
    df['status'] = df['status'].astype(str).str.strip()
    
    # Условия для замены
    is_empty = (df['status'] == '') | (df['status'].str.lower() == 'nan')
    starts_with_prop = df['status'].str.lower().str.startswith('проп')
    starts_with_stop = df['status'].str.lower().str.startswith('стоп')
    starts_with_dead = df['status'].str.lower().str.startswith(('погиб', '(погиб)'))
    starts_with_alive = df['status'].str.lower().str.startswith('жив')
    
    # Объединенные маски
    prop_mask = is_empty | starts_with_prop | starts_with_stop
    
    # Заменяем значения (теперь без учета gender)
    df.loc[prop_mask, 'status'] = 'пропал(а)'
    df.loc[starts_with_dead, 'status'] = 'погиб(ла)'
    df.loc[starts_with_alive, 'status'] = 'жив(а)'
    
    # Сохранение результата
    df.to_csv('updated_file.csv', index=False)
    
    print("Обработка завершена. Статистика замен:")
    print(df['status'].value_counts(dropna=False))

Обработка завершена. Статистика замен:
status
пропал(а)    33305
жив(а)        5516
погиб(ла)     1627
Name: count, dtype: int64
